
**Nombre**: Nicolás Alvarado.

# **Actividad 1** 

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta**: Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?

Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?

¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)

¿Qué pasaría si se agrega un nuevo rating a la base de datos?

Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

**Respuesta**: Slope One es de CF ya que usa los ratings generados por los usuarios que han consumido items semejantes. Además está basado en items ya que usa diferencia entre los promedios de los elementos. 

Slope One recibe los ratings de los usuarios y obtiene el promedio de los ratings por item. Por otra parte, si agregamos un nuevo rating a la base de datos, hay que volver a calcular el promedio.

# **Configuración Inicial**

## **Paso 1**:

Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:

In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    736      0 --:--:-- --:--:-- --:--:--   736
100 1546k  100 1546k    0     0  2251k      0 --:--:-- --:--:-- --:--:-- 2251k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    839      0 --:--:-- --:--:-- --:--:--   838
100  385k  100  385k    0     0   641k      0 --:--:-- --:--:-- --:--:--  641k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1306      0 --:--:-- --:--:-- --:--:--  1306
100  230k  100  230k    0     0   512k      0 --:--:-- --:--:-- --:--:--  512k


Los archivos u1.base y u1.test tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset movielens.

## **Paso 2**:

Instalamos pyreclab utilizando pip.**texto en negrita**

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 4.9 MB/s 


## **Paso 3**:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**
## **Paso 4**:
Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**
## **Paso 5**:
Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

# **Actividad 2** 

**Pregunta**: Explique qué hace el método train() en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos?

**Respuesta**: El método train() calcula las medias y ordena los datos.

## **Paso 6**:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [12]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


# **Actividad 3**

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta**: ¿Cómo se explican estos resultados?

**Respuesta**: Existe una diferencia considerable en las predicciones (4.5 vs 4.1 aproximadamente). Esto tiene sentido ya que no hay recomendaciones comunes.

In [13]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## **Paso 7**:
Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.

In [14]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [15]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

# **Actividad 4** 
Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

Pregunta: ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

Respuesta: Usando el usuario 53 y la recomendación Most popular (con topN=30) vemos que existe una recomendación bastante diferente. De hecho, al parecer, solo coincide Terminator 2 como película recomendada.

In [16]:
most_popular = pyreclab.MostPopular(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)  
most_popular.train()

In [17]:
recmovies_most_popular = most_popular.recommend("53", 30)
recmovies_most_popular = np.array(recmovies_most_popular).astype(int)

In [19]:
info_file.loc[recmovies_most_popular]['title']

movieid
50                                 Star Wars (1977)
100                                    Fargo (1996)
258                                  Contact (1997)
181                       Return of the Jedi (1983)
294                                Liar Liar (1997)
286                     English Patient, The (1996)
288                                   Scream (1996)
1                                  Toy Story (1995)
121                   Independence Day (ID4) (1996)
300                            Air Force One (1997)
127                           Godfather, The (1972)
7                             Twelve Monkeys (1995)
98                 Silence of the Lambs, The (1991)
172                 Empire Strikes Back, The (1980)
56                              Pulp Fiction (1994)
237                            Jerry Maguire (1996)
204                       Back to the Future (1985)
313                                  Titanic (1997)
173                      Princess Bride, The (1987)
405 

In [18]:
new_recommendation = mySlopeOne.recommend("53", 30)
print('Lista de items según ID:', new_recommendation)
new_recmovies_slopeone = np.array(new_recommendation).astype(int)
info_file.loc[new_recmovies_slopeone]['title']

Lista de items según ID: ['1653', '1064', '1642', '1639', '1599', '1512', '1450', '1536', '1500', '1463', '1449', '1398', '1201', '814', '113', '112', '111', '110', '109', '108', '107', '106', '104', '102', '101', '97', '96', '95', '94', '92']


movieid
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1642                             Some Mother's Son (1996)
1639                  Bitter Sugar (Azucar Amargo) (1996)
1599                        Someone Else's America (1995)
1512               World of Apu, The (Apur Sansar) (1959)
1450                               Golden Earrings (1947)
1536                                 Aiqing wansui (1994)
1500                            Santa with Muscles (1996)
1463                                     Boys, Les (1997)
1449                               Pather Panchali (1955)
1398                                          Anna (1996)
1201           Marlene Dietrich: Shadow and Light (1996) 
814                         Great Day in Harlem, A (1994)
113     Horseman on the Roof, The (Hussard sur le toit...
112                                        Flipper (1996)
111                   Truth About Cats & Dogs, The (1996)
110   


# **Actividad 5**

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta**: Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

Respuesta: En esta parte, vemos que la recomendación para el usuario 44 es exactamente la misma ya sea usando Most Popular o Slope One.

In [24]:
ranking = [int(r) for r in most_popular.recommend("44", 10, includeRated=False)]
print('Recommendation for user {}: {}'.format("44", ranking))
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Recommendation for user 44: [50, 100, 181, 286, 1, 121, 300, 127, 7, 98]
Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object

In [25]:
reclist_slopeone = mySlopeOne.recommend("44", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object